In [24]:
!ls /kaggle/input/

ds-project-1-pu-aids-2024-28


In [25]:
import os
os.listdir("/kaggle/input/ds-project-1-pu-aids-2024-28")

['DS-Project-1-PU-AIDS-2024-28-sec-b']

In [26]:
import os
os.listdir("/kaggle/input/ds-project-1-pu-aids-2024-28/DS-Project-1-PU-AIDS-2024-28-sec-b")

['.gitignore',
 'ds-project-1.ipynb',
 'eigen_vectors.npy',
 'requirements.txt',
 'college_student_placement_dataset.csv',
 'likelihood_distribution_params.pkl']

In [31]:
import numpy as np
import pickle
import scipy.stats as s
from fastapi import FastAPI, Query
from typing import Annonated
from pydantic import BaseModel

def determine_placement_posterior_probability(input_features):

    # Convert input to numpy array
    input_features = np.array(input_features)
    input_features = input_features.reshape(1, input_features.shape[0])

    # Load eigen vectors
    eig_vectors = np.load("eigen_vectors.npy")

    # Project input features
    new_input_features = np.matmul(input_features, eig_vectors)

    placement_equals_1_likelihood = 1.0
    placement_equals_0_likelihood = 1.0

    # Load likelihood distribution parameters
    with open("likelihood_distribution_params.pkl", "rb") as file_handle:
        likelihood_distribution_params = pickle.load(file_handle)

    # Priors (example – adjust based on your dataset)
    placement_equals_0_prior = 0.5
    placement_equals_1_prior = 0.5

    # Iterate over features
    for i, input_feat_value in enumerate(new_input_features[0]):

        mu_0, sigma_0 = likelihood_distribution_params[0][i]
        mu_1, sigma_1 = likelihood_distribution_params[1][i]

        p_input_feature_on_0_placement = s.norm.pdf(input_feat_value, mu_0, sigma_0)
        p_input_feature_on_1_placement = s.norm.pdf(input_feat_value, mu_1, sigma_1)

        placement_equals_0_likelihood *= p_input_feature_on_0_placement
        placement_equals_1_likelihood *= p_input_feature_on_1_placement

    placement_equals_0_posterior = placement_equals_0_likelihood * placement_equals_0_prior
    placement_equals_1_posterior = placement_equals_1_likelihood * placement_equals_1_prior

    if placement_equals_1_posterior > placement_equals_0_posterior:
        return {
            "result": f"Given your inputs, most likely you are going to get placed and the probability is roughly {placement_equals_1_posterior}"
        }
    else:
        return {
            "result": f"Given your inputs, most likely you are not going to get placed and the probability is roughly {placement_equals_1_posterior}"
        }
    class InputFeatureVector(BaseModel):
            iq: Annotated[int, Query(ge=40,le=160)]
            previous_semester_result: Annotated[float, Query(ge=0,le=10)]
            cgpa: Annotated[float, Query(ge=0,le=10)]
            communication_skills: Annotated[int, Query(ge=0,le=10)]
            projects_completed: Annotated[int, Query(ge=0,le=10)]

        app = FastAPI()
        @app.get("/")
        def home_page():

            return "This Web API tells the Probability of a student getting placed, based on his/her IQ,Previous Semester Result,CGPA,Communication Skills,Projects Completed"

        @app.post("/compute-probability")
        def compute_probability(input_features:InputFeatureVector):
            pass


In [28]:
NEW_DATA_INPUT_FEATURE_NAMES = ["Input_feat_1","Input_feat_2","Input_feat_3","Input_feat_4","Input_feat_5"]
PLACEMENT_EQUALS_1_PRIOR = 0.160